In [6]:
pip install --upgrade --force-reinstall scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 5.9 MB/s eta 0:00:0000:01m00:01
  Using cached numpy-1.26.1-cp310-cp310-macosx_10_9_x86_64.whl (20.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.1
    Uninstalling numpy-1.26.1:
      Successfully uninstalled numpy-1.26.1
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.0
    Uninstalling scipy-1.10.0:
      Successfully uninstalled scipy-1.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt 

in_data = loadmat('face_emotion_data.mat')
print([key for key in in_data]) # -- use this line to see the keys in the dictionary data structure 

# m(faces) x n(features) = y

['__header__', '__version__', '__globals__', 'y', 'X']


In [12]:
#1a)
y=in_data["y"]
X=in_data["X"]
# Using the formula: w = (X^T X)^(-1) X^T y
X_transpose = in_data["X"].transpose()
w = np.linalg.inv(X.transpose()@ X) @ X.transpose() @ y
w

array([[ 0.94366942],
       [ 0.21373778],
       [ 0.26641775],
       [-0.39221373],
       [-0.00538552],
       [-0.01764687],
       [-0.16632809],
       [-0.0822838 ],
       [-0.16644364]])

In [13]:
#1b)
# Each weight corresponds to one of the 9 features the model takes
# when we solve y=X^Tw, each weight will be applied to its associated measurement for all 128 faces

In [14]:
#1c)
#The features that seem to be most important are the ones who have the highest associated weights. Especially since
#all the features have been normalized, they are on the same scale. Features x1, x3 and x4 seem to be the most important.

In [15]:
#1d)
selected_columns = [0, 2, 3]
x_slice = X[:, selected_columns]
w2 = np.linalg.inv(x_slice.transpose()@ x_slice) @ x_slice.transpose() @ y
#If we are minimizing the features we want to use we should include the ones that have the most importance. These being
#the three expressed above
w2

array([[ 0.70546316],
       [ 0.8737872 ],
       [-0.78805643]])

In [16]:
#1e)
y_hat1 = np.sign(X@w)
y_hat2 = np.sign(x_slice@w2)

error_vec1 = [0 if i[0]==i[1] else 1 for i in np.hstack((y_hat1, y))]
error_vec2 = [0 if i[0]==i[1] else 1 for i in np.hstack((y_hat2, y))]

print("Percent error for 9 features: {}%. Percent error for 3 features: {}%.".format(round(sum(error_vec1)/128*100, 2), round(sum(error_vec2)/128*100, 2)))

Percent error for 9 features: 2.34%. Percent error for 3 features: 6.25%.


In [19]:
#1f)
num_subsets = 8
subset_size = len(X) // num_subsets
error_rates = []

#8folds cross validation
for fold in range(num_subsets):
    start_index = fold * subset_size
    end_index = (fold + 1) * subset_size

    X_train = np.concatenate((X[:start_index], X[end_index:]), axis=0)
    y_train = np.concatenate((y[:start_index], y[end_index:]), axis=0)
    X_holdout = X[start_index:end_index]
    y_holdout = y[start_index:end_index]

    predictions = X_holdout @ w
    misclassifications = np.sum(np.sign(predictions) != y_holdout)
    error_rate = misclassifications / len(X_holdout)
    error_rates.append(error_rate)


average_error_rate = np.mean(error_rates)
print("Average Error Rate: {}%.".format(round(average_error_rate*100, 2)) )

Average Error Rate: 2.34%.
